In [ ]:
Import-Module ImportExcel -ea stop -passthru | Join-String { $_.Name, $_.Version } -sep ', '  ; ('last update: {0}' -f @((get-date).tostring('u')))

ImportExcel 7.6.0
last update: 2022-07-03 22:23:39Z


In [ ]:
$Config = @{
    ExportRoot = 'C:\Temp\ImportExcel'
    EnableAutoOpenLast = $true
}
if( -not $Config.ExportRoot ) { 
    # mkdir -path $Config.ExportRoot  #-Confirm # can't actually input in notebooks?
    $Config.ExportRoot = Gi -ea stop $Config.ExportRoot
}
# ls $Config.ExportRoot

function relPath {
    #sugar for relative filepaths
    [CmdletBinding()]
    param(
        # Files, not folders
        [Alias('Name', 'RelPath')]
        [Parameter(Mandatory, Position=0)]$Path,

        # Files, not folders
        [Alias('Type')]
        [ArgumentCompletions('Csv', 'xlsx', 'md', 'ps1')]
        [Parameter(Position=1)]$Extension = 'xlsx'
    )
        if( -not $Config.ExportRoot ) { 
            write-error 'Missing Export Path in $Config.ExportRoot'
            return
            # do throws break the nteract kernel
            # throw 'Missing Export Path in $Config.ExportRoot' #
        }
        $script:__nbState ??= @{ }
        $sheetName = '{0}.{1}' -f @( $Path ; $Extension   )
        $fInfo = [IO.FileInfo]::new( (Join-Path $Config.ExportRoot $sheetName ) )        
        $script:__nbState.LastFile = $finfo
        
        $finfo | write-information
        return $finfo
}
function openLast {
    # Sugar for boilerpoint to minize code in cells
    # Note: Export-Excel has param "-Show"
    param()

    $Path = gi -ea stop $script:__nbState.LastFile 
    "Reading '$($Path.FullName)'..."    
    if(-not $Config.EnableAutoOpenLast) {write-error '$Config has auto-open disabled' ;return}
    $Path | select -first 1 | ii 
}
# relPath 'test' | % fullname


ImportExcel 7.6.0
last update: 2022-07-03 22:23:23Z


<a name='nb-top' href='#'>Top</a>
This is a summary of files in the repo [github.com/dfinke/ImportExcel](https://github.com/dfinke/ImportExcel)

<!-- #- <a href='#nb-item-1'>Empty File</a> -->

<a href='#nb-top'>Go Top</a>
- [Plot](https://github.com/dfinke/ImportExcel/blob/master/Examples/Charts/plot.ps1)

In [ ]:
& {
    function plot {
        param(
            $f,
            $minx,
            $maxx
        )

        $minx=[math]::Round($minx,1)
        $maxx=[math]::Round($maxx,1)

        #Get rid of pre-exisiting sheet
        $xlSourcefile = Relpath 'simplePlot'
        Write-Verbose -Verbose -Message  "Save location: $xlSourcefile"
        Remove-Item $xlSourcefile -ErrorAction Ignore

    # $c = New-ExcelChart -XRange X -YRange Y -ChartType Line -NoLegend -Title Plot -Column 2 -ColumnOffSetPixels 35

        $(for ($i = $minx; $i -lt $maxx-.1; $i+=.1) {
            [pscustomobject]@{
                X=$i.ToString("N1")
                Y=(&$f $i)
            }
        }) | Export-Excel $xlSourcefile -Show -AutoNameRange -LineChart -NoLegend  #-ExcelChartDefinition $c
    }

    function pi {[math]::pi}

    plot -f {[math]::Tan($args[0])} -minx (pi) -maxx (3*(pi)/2-.01)
}

VERBOSE: Save location: C:\Temp\ImportExcel\simplePlot.xlsx
